In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Only in drive or if the git repo has not been cloned locally
# !git clone https://github.com/syrgkanislab/dynamic_synthetic_controls.git
# cd dynamic_synthetic_controls/

In [ ]:
# Add the folder to the import paths; required for importing during parallel
# for loops.
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('pcr.py')))

# PCR Implementation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
from pcr import PCR

# Simple: Prediction from Noisy Low-Rank Measurements

In [ ]:
N = 1000
T = 5000
u = np.random.normal(0, 1, size=(N, 1))
v = np.random.normal(0, 1, size=(T, 1))
W = u @ v.T
Z = W + np.random.normal(0, .7, size=(N, T))

In [ ]:
X = Z[1:, :-10].T
y = Z[0, :-10]
Xtest = Z[1:, -10:].T
ytest = Z[0, -10:]
gtest = W[0, -10:]

In [ ]:
cross_val_score(PCR(), X, y, scoring='r2')

In [ ]:
est = PCR().fit(X, y)

In [ ]:
pred_test = est.predict(Xtest)

In [ ]:
plt.plot(pred_test, label='pred')
plt.plot(ytest, label='Y')
plt.plot(gtest, label='E[Y]')
plt.legend()
plt.show()

In [ ]:
plt.hist(est.coef_)
plt.show()

# Synthetic Controls with Staggered Rollout

In [ ]:
N = 1000 # n units
T = 5000 # n overall time periods
K = 2 # n actions (for now has to be true)
u = np.random.normal(0, 1, size=(N, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jtk->itk', u, v.T) # true mean potential outcomes for each unit and period
Z = W + np.random.normal(0, .5, size=(N, T, K)) # random potential outcomes for each unit and period

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=N, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (N, 1)) # helper matrix
A = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * 1 # set treatment to 1 after rollout

In [ ]:
Zobs = Z[:, :, 0] * (1 - A) + Z[:, :, 1] * A # observed noisy outcomes
Wobs = W[:, :, 0] * (1 - A) + W[:, :, 1] * A # observed true mean outcomes

In [ ]:
# we only care about the first unit and we use the rest to predict
X = Zobs[1:, :T0].T
y = Zobs[0, :T0]
Xtest = Zobs[1:, T0:].T
ytest = Zobs[0, T0:]

In [ ]:
t0[0] - T0

In [ ]:
pred_test = np.zeros(T)
for t in np.arange(T0, T):
    donors = (A[1:, t] == 0) # find units that are un-treated in this post-treatment period
    est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
    pred_test[t] = est.predict(Xtest[t - T0, donors]) # predict the outcome for the target unit for this period

In [ ]:
t1 = T0
t2 = T
plt.plot(pred_test[t1:t2], label='pred(0)')
# plt.plot(ytest, label='observed')
plt.plot(Wobs[0, t1:t2], label='E[Y]')
plt.plot(W[0, t1:t2, 0], label='E[Y(0)]')
plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
plt.xticks(ticks=np.arange(t2 - t1), labels=A[0, t1:t2])
plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
plt.xlabel('treatment per period')
plt.legend()
plt.show()

# Synthetic Interventions

In [ ]:
N = 500 # n units
T = 5000 # n overall time periods
K = 3 # n actions (for now has to be true)
u = np.random.normal(0, 1, size=(N, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jtk->itk', u, v.T) # true mean potential outcomes for each unit and period
Z = W + np.random.normal(0, 2, size=(N, T, K)) # random potential outcomes for each unit and period

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=N, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (N, 1)) # helper matrix
A = np.random.choice(np.arange(1, K), size=(N, T), replace=True)
A = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * A # set treatment to 1 after rollout

In [ ]:
Zobs = Z[:, :, 0] * (A == 0) # building observed noisy outcomes
Wobs = W[:, :, 0] * (A == 0) # building observed true mean outcomes
for t in np.arange(1, K):
    Zobs += Z[:, :, t] * (A == t) 
    Wobs += W[:, :, t] * (A == t)

In [ ]:
# we only care about the first unit and we use the rest to predict
X = Zobs[1:, :T0].T
y = Zobs[0, :T0]
Xtest = Zobs[1:, T0:].T
ytest = Zobs[0, T0:]

In [ ]:
t0[0] - T0

In [ ]:
# calculate mean counterfactual outcome for each period and each potential treatment
pred_test = np.zeros((K, T))
for k in np.arange(K):
    for t in np.arange(T0, T):
        donors = (A[1:, t] == k) # find units that received treatment k in this post-treatment period
        est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
        pred_test[k, t] = est.predict(Xtest[t - T0, donors]) # predict the outcome for the target unit for this period

In [ ]:
t1 = T0
t2 = T
for k in range(K):
    plt.figure(figsize=(15, 5))
    plt.plot(pred_test[k, t1:t2], label=f'pred({k})')
    plt.plot(W[0, t1:t2, k], label=f'E[Y({k})]')
    plt.xticks(ticks=np.arange(t2 - t1), labels=A[0, t1:t2])
    plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
    plt.xlabel('treatment per period')
    plt.legend()
    plt.show()

# Synthetic Blips

We generate potential outcomes as follows:
\begin{align}
Y_{n,t}(a) :=~& \sum_{\ell=0}^{L} \langle u_{n}, v_{t, \ell, a_{t-\ell}} \rangle + \epsilon_{n, t, \ell, a_{t-\ell}} &
\epsilon_{n, t, \ell, a_{t-\ell}}\sim N(0, \sigma^2)
\end{align}
where $n\in [N]$ denotes a unit, $t\in [T]$ a time period and $\ell\in [L]$ a "lag". Then the observed data are $Y_{n,t} := Y_{n,t}(A_{n,1}, \ldots, A_{n,t})$. 

We will denote with:
\begin{align}
W_{n, t, \ell, a} :=~& \langle u_{n}, v_{t, \ell, a} \rangle &
Z_{n, t, \ell, a} :=~& \langle u_{n}, v_{t, \ell, a} \rangle + \epsilon_{n, t, \ell, a_{t-\ell}}\\
\end{align}
Then we have that:
\begin{align}
Y_{n,t}(a) =~& \sum_{\ell=0}^{L} Z_{n, t, \ell, a_{t-\ell}} &
E[Y_{n,t}(a)] =~& \sum_{\ell=0}^{L} W_{n, t, \ell, a_{t-\ell}}
\end{align}
Moreover, the blip effects are of the form:
\begin{align}
\gamma_{n,t,t-\ell}(a) := W_{n, t, \ell, a} - W_{n, t, \ell, 0} = \langle u_n, v_{t, \ell, a} - v_{t,\ell, 0}\rangle
\end{align}
and the baseline effect takes the form:
\begin{align}
b_{n, t}(a) := \sum_{\ell=0}^L W_{n, t, \ell, 0} = \langle u_n, \sum_{\ell=0}^L v_{t, \ell, 0} \rangle
\end{align}

In [ ]:
N = 500 # n units
T = 5000 # n overall time periods
K = 3 # n actions (for now has to be true)
L = 1 # number of lags that impact current outcome
u = np.random.normal(0, 1, size=(N, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, L + 1, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jltk->iltk', u, v.T) # true mean potential blips for each unit and period and lag
Z = W + np.random.normal(0, .4, size=(N, T, L + 1, K)) # random potential blips for each unit, period and lag

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=N, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (N, 1)) # helper matrix
A = np.random.choice(np.arange(1, K), size=(N, T), replace=True)
A = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * A # set treatment to 1 after rollout

In [ ]:
Zobs = np.zeros(Z.shape[:2]) # building observed noisy outcomes
Wobs = np.zeros(W.shape[:2]) # building observed true mean outcomes
for ell in range(L + 1): # for each lag period
    Aell = np.roll(A, ell) # we find the lag treatment for each period
    Aell[:, :ell] = 0
    for k in range(K):
        Zobs += Z[:, :, ell, k] * (Aell == k) # we add the lag blip effect of that lag treatment
        Wobs += W[:, :, ell, k] * (Aell == k) # we add the lag blip effect of that lag treatment

In [ ]:
def get_true_counterfactual(unit, Atarget):
    '''Counterfactual post-T0 mean potential outcome trajectories for unit `unit`;
    `Atarget` contains target treatments in periods {T0, ..., T}.'''
    Wtarget = np.zeros(T - T0) * np.nan
    t1 = T0 + L
    for t in np.arange(t1, T):
        Wtarget[t - T0] = 0
        for ell in range(L + 1): # for each lag period
            for k in range(K):
                # we add the lag blip effect of that lag treatment
                Wtarget[t - T0] += W[unit, t, ell, k] * (Atarget[t - T0 - ell] == k)
    return Wtarget

## Synthetic Blip Algorithm

### Calculation of all weights

We first calculate for every unit $n$, for every action $a$ and for every time step $t >= T0$, the $N$-dimensional vector of donor weights: $\beta_{n, t}^{I_a}$ using PCR (note that $\beta_{n,t}^{I_a}$ is only supported on the set of donors $I_a$, but for code convenience we embded it in an $N$ dimensional space, with the non-donor coordinates being $0$). Thus we call PCR $N \times (T-T0) \times K$ times. We do this in parallel over the $N$ target units.

In [ ]:
from joblib import Parallel, delayed

def donor_weights(i):
    X = Zobs[:, :T0].T
    y = Zobs[i, :T0]
    # calculate mean counterfactual outcome for each period and each potential treatment
    Beta = np.zeros((K, T - T0, N))
    for k in np.arange(K):
        for t in np.arange(T0, T):
            # find units that received treatment k in period t, as their first treatment in this post-treatment period
            donors = (A[:, t] == k) & np.all(A[:, :t] == 0, axis=1)
            est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
            Beta[k, t - T0, donors] = est.coef_ # store the unit weights in the matrix Beta
    return Beta

# The matrix Beta will be of shape (n, K, T - T0, n). Each entry (i, k, t, :)
# will contain the donor weights with target unit i, among donors which received
# treatment k, as their first treatment and at period t.
Beta = np.array(Parallel(n_jobs=-1, verbose=3)(delayed(donor_weights)(i) for i in range(N)))

In [ ]:
Beta.shape # (N, K, T - T0, N)

### Calculating synthetic blips

Now that we have the donor weights, we calculate the synthetic baseline response for every target unit $n$ and period $t$. Moreover, we calculate in a recursive manner, the synthetic blip effects $\gamma_{n, t, t-\ell}(a)$ for every target unit $n$, for every time step $t\geq T0 + L$, and for every lag $\ell \in \{0, \ldots, L\}$ and for every action $a\in [K]$.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

base = np.nan * np.zeros((N, T)) # baseline response for each unit and period
blip = np.nan * np.zeros((N, T, L + 1, K)) # blip effects for each unit i, period t, lag ell and action k

base_var = np.nan * np.zeros((N, T)) # TRIAL
blip_var = np.nan * np.zeros((N, T, L + 1, K)) # TRIAL

# We start producing counterfactuals starting from period T0 + L, as for earlier periods
# we don't have enough "lags"
for t in np.arange(T0 + L, T):

    # for each post intervention period and unit, estimate the mean baseline response
    base[:, t] = Beta[:, 0, t - T0, :] @ Zobs[:, t] # sum_{j\in I_t^0} \beta_j^{i, I_t^0} Y_{j, t}
    
    ### DRAFT ATTEMPT: Variance of estimate calculation
    donors = np.abs(Beta[:, 0, t - T0, :]) > 0
    var_base = np.mean(np.tile((Zobs[:, t] - base[:, t])**2, (N, 1)), axis=1, where=donors)
    base_var[:, t] = var_base * np.sum(Beta[:, 0, t - T0, :]**2, axis=1)
    ###################################################

    for ell in range(L + 1):  # we construct the blip effects for lag ell
        for k in range(K):   # and for each action k, i.e. gamma_{j, t, t-ell}(k)

            # we build the "obesrved" blip effects; we will actually for a moment pretend that
            # every unit is in the I_{t-ell}^k, but then all the "wrong" entries will be corrected
            # by taking the inner product with the donor entries and since donor weights will only
            # be supported on elements in I_{t-ell}^k. This is more convenient for coding purposes
            observed_blips = Zobs[:, t] - base[:, t] # we subtract the baseline response Y_{j, t} - b_{j, t}

            for ellp in range(ell): # for each smaller lag, i.e. period t - ellp, with ellp < ell
                # we subtract the blip effect of the treatment that each unit received at period t - ellp
                # this subtracts gamma_{j, t, t - ellp}(A_{j, t - ell})
                ohe = OneHotEncoder(sparse=False, categories=[np.arange(K)])
                lagAohe = ohe.fit_transform(A[:, [t - ellp]]) # this is the treatment at t-ellp
                observed_blips -= np.sum(blip[:, t, ellp, :] * lagAohe, axis=1)

            # now that we have constructed the observed blip effects for all donor units
            # we can impute the blip effects for all units, using the donor weights
            # we will in fact even replace the blip effects of the donor units, with their
            # corresponding averages, which will induce variance reduction
            blip[:, t, ell, k] = Beta[:, k, t - ell - T0, :] @ observed_blips

            ### DRAFT ATTEMPT: Variance of estimate calculation
            donors = np.abs(Beta[:, k, t - ell - T0, :]) > 0
            donor_wsq = Beta[:, k, t - ell - T0, :]**2
            # variance of current blip effect
            var_blips = np.mean(np.tile((observed_blips - blip[:, t, ell, k])**2, (N, 1)), axis=1, where=donors)
            blip_var[:, t, ell, k] = var_blips * np.sum(donor_wsq, axis=1)
            # influence from variance of future blip effect estimation and base response estimation
            # TODO. This influence part is wrong. The different future blip and base quantities
            # are not independent variables with each other across units; this formula treats them as such. 
            blip_var[:, t, ell, k] += donor_wsq @ base_var[:, t]
            for ellp in range(ell):
                ohe = OneHotEncoder(sparse=False, categories=[np.arange(K)])
                lagAohe = ohe.fit_transform(A[:, [t - ellp]]) # this is the treatment at t-ellp
                blip_var[:, t, ell, k] += donor_wsq @ np.sum(blip_var[:, t, ellp, :] * lagAohe, axis=1)
            ###################################################

### Sanity Checks

In [ ]:
base[0, -1], np.sqrt(base_var[0, -1]), np.sum(W[0, -1, :, 0]) # baseline response

In [ ]:
blip[0, -1, :, 0] # should be zero

In [ ]:
blip[0, -1, :, 1], W[0, -1, :, 1] - W[0, -1, :, 0] # matching true blip effects

In [ ]:
np.sqrt(blip_var[0, -1, :, 1])

In [ ]:
blip[0, -1, :, 2], W[0, -1, :, 2] - W[0, -1, :, 0] # matching true blip effects

In [ ]:
np.sqrt(blip_var[0, -1, :, 2])

### Plot Counterfactual Action Sequence Predictions and True Values

In [ ]:
def get_pred_counterfactual(unit, Atarget):
    pred = np.zeros(T - T0) * np.nan
    pred_var = np.zeros(T - T0) * np.nan
    t1 = T0 + L
    for t in np.arange(t1, T):
        pred[t - T0] = base[unit, t]
        pred_var[t - T0] = base_var[unit, t]
        for ell in range(L + 1): # for each lag period
            for k in range(K):
                # we add the lag blip effect of that lag treatment
                pred[t - T0] += blip[unit, t, ell, k] * (Atarget[t - T0 - ell] == k)
                pred_var[t - T0] += blip_var[unit, t, ell, k] * (Atarget[t - T0 - ell] == k)

    return pred, pred_var

In [ ]:
Atarget = np.random.choice(np.arange(1, K), size=(T - T0), replace=True)

plt.figure(figsize=(15, 5))
# plt.plot(ytest, label='observed')
xgrid = np.arange(T - T0)
pred, pred_var = get_pred_counterfactual(0, Atarget)
plt.plot(pred, label=f'pred(a)')
plt.fill_between(xgrid, pred - 1.96 * np.sqrt(pred_var), pred + 1.96 * np.sqrt(pred_var), alpha=.4)
plt.plot(get_true_counterfactual(0, Atarget), label=f'E[Y(a)]')
plt.xticks(ticks=np.arange(T - T0), labels=Atarget)
plt.axvline(0, color='magenta', linestyle='--')
plt.xlabel('treatment per period')
plt.legend()
plt.show()

In [ ]:
t1 = T0
t2 = T
for k in range(K):
    Atarget = np.ones(t2 - t1) * k
    plt.figure(figsize=(15, 5))
    pred, pred_var = get_pred_counterfactual(0, Atarget)
    plt.plot(pred, label=f'pred({k},...,{k})')
    plt.fill_between(xgrid, pred - 1.96 * np.sqrt(pred_var), pred + 1.96 * np.sqrt(pred_var), alpha=.4)
    plt.plot(get_true_counterfactual(0, Atarget), label=f'E[Y({k},...,{k})]')
    plt.xticks(ticks=np.arange(t2 - t1), labels=Atarget)
    plt.axvline(0, color='magenta', linestyle='--')
    plt.xlabel('treatment per period')
    plt.legend()
    plt.show()

## Packaging Data Gen in a Class

In [ ]:
N = 50 # n units
T = 1000 # n overall time periods
K = 2 # n actions (for now has to be true)
L = 1 # number of lags that impact current outcome
T0 = T - 10 # pre-treatment period

In [ ]:
from gen_data import SynthDGP

dgp = SynthDGP(N, T, K, L, T0)
dgp.init_instance()

In [ ]:
W, Z, Wobs, Zobs, A = dgp.sample_data()

## Packaging in an Estimator Class

In [ ]:
from synthblip import SyntheticBlip

synth = SyntheticBlip().fit(Zobs, A, T0, K, L)

In [ ]:
synth.base_[0, -1], np.sqrt(synth.base_var_[0, -1]), np.sum(W[0, -1, :, 0]) # baseline response

In [ ]:
synth.blip_[0, -1, :, 0] # should be zero

In [ ]:
synth.blip_[0, -1, :, 1], W[0, -1, :, 1] - W[0, -1, :, 0] # matching true blip effects

In [ ]:
np.sqrt(synth.blip_var_[0, -1, :, 1])

In [ ]:
Atarget = np.random.choice(np.arange(1, K), size=(T - T0), replace=True)

plt.figure(figsize=(15, 5))
# plt.plot(ytest, label='observed')
xgrid = np.arange(T - T0)
pred, pred_var = synth.predict_counterfactual(0, Atarget)
plt.plot(pred, label=f'pred(a)')
plt.fill_between(xgrid, pred - 1.96 * np.sqrt(pred_var), pred + 1.96 * np.sqrt(pred_var), alpha=.4)
plt.plot(dgp.get_counterfactual(0, Atarget), label=f'E[Y(a)]')
plt.xticks(ticks=np.arange(T - T0), labels=Atarget)
plt.axvline(0, color='magenta', linestyle='--')
plt.xlabel('treatment per period')
plt.legend()
plt.show()

## Coverage Experiments

In [ ]:
N = 500 # n units
T = 1000 # n overall time periods
K = 2 # n actions (for now has to be true)
L = 1 # number of lags that impact current outcome
T0 = T - 10 # pre-treatment period

In [ ]:
from gen_data import SynthDGP

dgp = SynthDGP(N, T, K, L, T0)
dgp.init_instance(random_seed=123)

In [ ]:
np.random.seed(123)
Atarget = np.random.choice(np.arange(1, K), size=(T - T0), replace=True)

In [ ]:
Wtarget = dgp.get_counterfactual(0, Atarget)

In [ ]:
from synthblip import SyntheticBlip

def exp(it):
    print(it)
    W, Z, Wobs, Zobs, A = dgp.sample_data(random_seed=it)
    synth = SyntheticBlip().fit(Zobs, A, T0, K, L)
    pred, pred_var = synth.predict_counterfactual(0, Atarget)
    return pred, pred_var

In [ ]:
results = [exp(it) for it in range(100)]

In [ ]:
preds, preds_var = zip(*results)

In [ ]:
preds = np.array(preds)
preds_var = np.array(preds_var)

In [ ]:
lower = preds - 1.96 * np.sqrt(preds_var)
upper = preds + 1.96 * np.sqrt(preds_var)

In [ ]:
coverage = (lower <= Wtarget) & (Wtarget <= upper)

In [ ]:
np.mean(coverage, axis=0)